# Calibration tracking demo

This notebook shows how to enable experiment tracking for the built-in calibration routines. The examples highlight both Weights & Biases and MLflow setups using the new `TrackingConfig` helper.

## 1. Configure your tracker

Install and authenticate the client (`pip install wandb` or `pip install mlflow`) and then pick the provider in the configuration.

In [ ]:
from neutryx.models import heston, sabr
from neutryx.utils.tracking import TrackingConfig, tracker_context
import jax.numpy as jnp

S0 = 100.0
strikes = jnp.array([90, 100, 110])
maturities = jnp.array([0.5, 1.0, 1.5])
target_prices = jnp.array([12.1, 8.5, 5.4])
target_iv = jnp.array([0.22, 0.21, 0.2])


### Weights & Biases example

The `tracker_context` helper ensures the run is closed even when an error occurs.

In [ ]:
wandb_cfg = TrackingConfig(
    enable=True,
    provider='wandb',
    project='neutryx-demo',
    experiment_name='heston-calibration',
    tags=['demo', 'calibration'],
    log_every=20,
)

with tracker_context(wandb_cfg) as tracker:
    _ = heston.calibrate(
        S0,
        strikes,
        maturities,
        target_prices,
        n_iterations=60,
        tracker=tracker,
        log_every=10,
    )


### MLflow example

Switching to MLflow is as simple as changing the provider. If you need a custom tracking URI supply it through `run_kwargs`.

In [ ]:
mlflow_cfg = TrackingConfig(
    enable=True,
    provider='mlflow',
    project='neutryx-demo',
    experiment_name='sabr-calibration',
    tags=['demo', 'calibration'],
    run_kwargs={'tracking_uri': 'file:mlruns'},
)

with tracker_context(mlflow_cfg) as tracker:
    _ = sabr.calibrate(
        F=S0,
        strikes=strikes,
        maturities=maturities,
        target_iv=target_iv,
        tracker=tracker,
        log_every=20,
    )


## 2. Custom logging templates

The helpers `calibration_param_template` and `calibration_metric_template` standardise output across trackers. You can call them directly when emitting custom diagnostics.

In [ ]:
from neutryx.utils.tracking import calibration_metric_template, calibration_param_template

params_payload = calibration_param_template({'alpha': 0.2, 'beta': 0.8}, prefix='custom')
metrics_payload = calibration_metric_template(0.0123, {'alpha': 0.19, 'beta': 0.82}, prefix='custom')
params_payload, metrics_payload
